In [ ]:
# Compare CAMx to TEMPO NO2, HCHO and FNR (HCHO/NO2)

In [ ]:
# READ THE control yaml file
import xarray as xr
an = driver.analysis()
an.control = 'control_tempo_camx.yaml'
an.read_control()

In [ ]:
# Read observations and models, pair data and generate paired output file
# or just read a paired outputfile if already exists
an.open_obs()
an.open_models()
an.pair_data()
an.save_analysis()
#an.read_analysis()

In [ ]:
# calculate FNR from NO2 and HCHO paired output
# only calculate FNR for pixels with NO2 > 1e15 molec/cm2
an.obs["tempo_l2_FNR"] = driver.observation()
an.obs["tempo_l2_FNR"].sat_type = "tempo_l2"
an.obs["tempo_l2_FNR"].variable_dict = {"tempo_l2_FNR": {"ylabel_plot": "FNR"}}

models = ["CAMX"]
no2 = ["NO2"]
hcho = ["FORM"]

for pair in an.paired.keys():
    an.paired[pair].obj = an.paired[pair].obj.sel(time=(an.paired[pair].obj['time'].dt.hour > 4))

for no2_m, hcho_m, model in zip(no2, hcho, models):
    ratio_FNR = xr.Dataset()
    ratio_FNR["tempo_l2_FNR"] = (
        an.paired[f"tempo_l2_hcho_{model}"].obj["vertical_column"]
        / an.paired[f"tempo_l2_no2_{model}"].obj["vertical_column_troposphere"]
    ).where(an.paired[f"tempo_l2_no2_{model}"].obj["vertical_column_troposphere"] > 1e15)
    ratio_FNR[f"{model}_FNR"] = (
        an.paired[f"tempo_l2_hcho_{model}"].obj[hcho_m]
        / an.paired[f"tempo_l2_no2_{model}"].obj[no2_m]
    ).where(an.paired[f"tempo_l2_no2_{model}"].obj[no2_m] > 1e15)   

    p = driver.pair()
    p.type = "sat_swath_clm"
    p.obs = "tempo_l2_FNR"
    p.model = f"{model}"
    p.model_vars = [f"{model}_FNR"]
    p.obs_vars = ["tempo_l2_FNR"]
    p.obj = ratio_FNR
    p.filename = f"tempo_l2_FNR_{model}.nc"
    an.paired[f"tempo_l2_FNR_{model}"] = p

In [ ]:
# plotting and statistics
an.plotting()
an.stats()